# Module 14: Ensemble Methods

**Estimated Time**: 90 minutes

## Learning Objectives

By the end of this module, you will:
- **Understand** the power of ensemble learning
- **Master** bagging techniques including Random Forests
- **Learn** boosting algorithms and when to use them
- **Apply** XGBoost, LightGBM, and CatBoost
- **Build** stacking and blending ensembles
- **Implement** Kaggle-winning techniques
- **Compare** ensemble methods on real datasets

## Prerequisites

- Modules 00-13 completed
- Understanding of decision trees
- Familiarity with cross-validation and hyperparameter tuning

## What are Ensemble Methods?

> **"In diversity there is strength"**

**Ensemble learning** combines multiple models to create a stronger predictor than any single model alone.

### The Wisdom of Crowds

Just like how:
- **Multiple doctors** provide better diagnoses than one
- **Jury decisions** are more reliable than individual judgments  
- **Team projects** leverage diverse skills

**Ensemble methods** leverage multiple models for better predictions!

### Why Ensembles Win

- **Kaggle competitions**: 90%+ of winners use ensembles
- **Production ML**: Most real-world systems use ensembles
- **Robustness**: Less sensitive to outliers and noise
- **Performance**: Often 5-15% better than single models

### Three Main Approaches

1. **Bagging** (Bootstrap Aggregating): Train same algorithm on different data subsets
2. **Boosting**: Train models sequentially, each correcting previous errors
3. **Stacking**: Combine different algorithms with a meta-model

Let's master them all!

---

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Ensemble methods
from sklearn.ensemble import (
    RandomForestClassifier,
    BaggingClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
    VotingClassifier,
    StackingClassifier,
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# Advanced ensemble libraries
try:
    import xgboost as xgb

    print("✓ XGBoost available")
except ImportError:
    print("⚠️  XGBoost not installed. Install with: pip install xgboost")
    xgb = None

try:
    import lightgbm as lgb

    print("✓ LightGBM available")
except ImportError:
    print("⚠️  LightGBM not installed. Install with: pip install lightgbm")
    lgb = None

try:
    from catboost import CatBoostClassifier

    print("✓ CatBoost available")
except ImportError:
    print("⚠️  CatBoost not installed. Install with: pip install catboost")
    CatBoostClassifier = None

import warnings

warnings.filterwarnings("ignore")

%matplotlib inline

# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

print("\n✓ Core ensemble libraries loaded!")
print("✓ Ready to build powerful ensembles!")

## 1. Ensemble Learning Introduction

Ensemble methods combine predictions from multiple models to improve accuracy and robustness.

### The Core Idea

**Single Model:**
```
Data → Model → Prediction
```

**Ensemble:**
```
Data → Model 1 ┐
Data → Model 2 ├→ Combine → Better Prediction
Data → Model 3 ┘
```

### Why Ensembles Work

1. **Reduce Variance** (Bagging)
   - Train on different data subsets
   - Average out individual model errors
   - Example: Random Forest

2. **Reduce Bias** (Boosting)
   - Focus on hard examples
   - Sequentially improve
   - Example: XGBoost, AdaBoost

3. **Improve Predictions** (Stacking)
   - Combine different model types
   - Learn optimal combination
   - Example: Stacking classifier

### Ensemble Taxonomy

| Method | Type | How It Works | Best For |
|--------|------|--------------|----------|
| **Random Forest** | Bagging | Multiple decision trees, different data | General purpose |
| **AdaBoost** | Boosting | Sequential weak learners, focus on errors | Binary classification |
| **Gradient Boosting** | Boosting | Sequential trees, gradient descent | Structured data |
| **XGBoost** | Boosting | Optimized gradient boosting | Kaggle, production |
| **LightGBM** | Boosting | Fast gradient boosting | Large datasets |
| **CatBoost** | Boosting | Handles categorical features | Categorical data |
| **Voting** | Meta-ensemble | Majority vote or average | Combining models |
| **Stacking** | Meta-ensemble | Meta-model learns combination | Maximum performance |

### When to Use Ensembles

✓ **Use when:**
- You need maximum accuracy
- Robustness is important
- You have computational resources
- Competition or production setting

✗ **Don't use when:**
- Interpretability is critical
- Very limited compute
- Real-time predictions needed (milliseconds)
- Model size matters (mobile deployment)

Let's load data and start building ensembles!

In [ ]:
# Load and Prepare Data

print("=" * 60)
print("PREPARING DATA FOR ENSEMBLE LEARNING")
print("=" * 60)

# Load feature engineering dataset
df = pd.read_csv("../../data_advanced/feature_engineering.csv")

# Prepare features
le_city = LabelEncoder()
le_job = LabelEncoder()
df["city_encoded"] = le_city.fit_transform(df["city"])
df["job_encoded"] = le_job.fit_transform(df["job_category"])

# Select features
features = [
    "age",
    "income",
    "education_years",
    "experience_years",
    "num_dependents",
    "city_encoded",
    "job_encoded",
]

X = df[features]
y = df["loan_approved"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nDataset: {X.shape[0]} samples, {X.shape[1]} features")
print(f"Train set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nClass distribution:")
print(y.value_counts())
print(f"Approval rate: {y.mean():.2%}")

# Baseline: Single Decision Tree
print("\n" + "=" * 60)
print("BASELINE: Single Decision Tree")
print("=" * 60)

baseline_tree = DecisionTreeClassifier(random_state=42)
baseline_tree.fit(X_train, y_train)

train_score = baseline_tree.score(X_train, y_train)
test_score = baseline_tree.score(X_test, y_test)

print(f"\nBaseline Decision Tree:")
print(f"  Train Accuracy: {train_score:.4f}")
print(f"  Test Accuracy:  {test_score:.4f}")
print(f"  Overfitting:    {train_score - test_score:.4f}")

if train_score - test_score > 0.1:
    print("\n⚠️  High overfitting! Decision trees overfit easily.")
    print("💡 Ensembles will help reduce this!")

print("\n✓ Data prepared and baseline established!")

## 2. Bagging and Random Forests

**Bagging** = **B**ootstrap **Agg**regat**ing**

### How Bagging Works

1. **Bootstrap**: Create N random samples (with replacement) from training data
2. **Train**: Train a model on each sample
3. **Aggregate**: Combine predictions (vote for classification, average for regression)

### Why It Reduces Variance

- Each model sees slightly different data
- Individual models make different errors
- Errors cancel out when averaged
- Final prediction is more stable

### Random Forest = Bagging + Random Feature Selection

**Standard Bagging:**
- Bootstrap samples
- All features considered

**Random Forest:**
- Bootstrap samples
- Random subset of features at each split
- Even more diversity!

### Key Hyperparameters

| Parameter | Description | Typical Values |
|-----------|-------------|----------------|
| `n_estimators` | Number of trees | 100-500 |
| `max_depth` | Tree depth | 10-30 or None |
| `min_samples_split` | Min samples to split | 2-10 |
| `max_features` | Features per split | 'sqrt', 'log2' |
| `max_samples` | Bootstrap sample size | 0.5-1.0 |

### Advantages

✓ Reduces overfitting  
✓ Handles high-dimensional data  
✓ Provides feature importance  
✓ Robust to outliers  
✓ Parallelizable  

### Disadvantages

✗ Can be slow  
✗ Memory intensive  
✗ Less interpretable  

Let's compare Bagging vs Random Forest!

In [ ]:
# Bagging and Random Forests Implementation

print("=" * 60)
print("BAGGING VS RANDOM FOREST")
print("=" * 60)

# 1. Standard Bagging
print("\n1. Bagging Classifier")
print("-" * 60)

bagging = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
    max_features=0.8,
    random_state=42,
    n_jobs=-1,
)

bagging.fit(X_train, y_train)
bag_train = bagging.score(X_train, y_train)
bag_test = bagging.score(X_test, y_test)

print(f"Train Accuracy: {bag_train:.4f}")
print(f"Test Accuracy:  {bag_test:.4f}")
print(f"Overfitting:    {bag_train - bag_test:.4f}")

# 2. Random Forest
print("\n2. Random Forest Classifier")
print("-" * 60)

rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
)

rf.fit(X_train, y_train)
rf_train = rf.score(X_train, y_train)
rf_test = rf.score(X_test, y_test)

print(f"Train Accuracy: {rf_train:.4f}")
print(f"Test Accuracy:  {rf_test:.4f}")
print(f"Overfitting:    {rf_train - rf_test:.4f}")

# 3. Comparison
print("\n" + "=" * 60)
print("COMPARISON SUMMARY")
print("=" * 60)

comparison = pd.DataFrame(
    {
        "Model": ["Single Tree", "Bagging", "Random Forest"],
        "Train Acc": [train_score, bag_train, rf_train],
        "Test Acc": [test_score, bag_test, rf_test],
        "Overfitting": [train_score - test_score, bag_train - bag_test, rf_train - rf_test],
    }
)

display(comparison)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy comparison
x = np.arange(len(comparison))
width = 0.35

axes[0].bar(x - width / 2, comparison["Train Acc"], width, label="Train", alpha=0.8)
axes[0].bar(x + width / 2, comparison["Test Acc"], width, label="Test", alpha=0.8)
axes[0].set_ylabel("Accuracy")
axes[0].set_title("Bagging Methods Comparison", fontweight="bold")
axes[0].set_xticks(x)
axes[0].set_xticklabels(comparison["Model"])
axes[0].legend()
axes[0].grid(axis="y", alpha=0.3)

# Overfitting comparison
colors = ["red" if x > 0.05 else "green" for x in comparison["Overfitting"]]
axes[1].barh(comparison["Model"], comparison["Overfitting"], color=colors, alpha=0.7)
axes[1].axvline(x=0.05, color="orange", linestyle="--", label="5% threshold")
axes[1].set_xlabel("Overfitting (Train - Test)")
axes[1].set_title("Overfitting Reduction", fontweight="bold")
axes[1].legend()
axes[1].grid(axis="x", alpha=0.3)

plt.tight_layout()
plt.show()

# Feature importance from Random Forest
print("\n" + "=" * 60)
print("FEATURE IMPORTANCE (Random Forest)")
print("=" * 60)

importance_df = pd.DataFrame(
    {"Feature": features, "Importance": rf.feature_importances_}
).sort_values("Importance", ascending=False)

print("\n" + importance_df.to_string(index=False))

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(importance_df["Feature"], importance_df["Importance"], color="steelblue", alpha=0.8)
plt.xlabel("Importance", fontweight="bold")
plt.title("Random Forest Feature Importance", fontweight="bold", fontsize=14)
plt.grid(axis="x", alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Key Insights:")
print("   • Bagging reduces overfitting significantly")
print("   • Random Forest adds feature randomization")
print("   • Both outperform single decision tree")
print("   • Feature importance helps interpretability")
print("\n✓ Bagging and Random Forests demonstrated!")

## 3. Boosting Fundamentals

**Boosting** trains models sequentially, each one correcting errors of the previous models.

### Key Difference: Bagging vs Boosting

**Bagging:**
```
Train in parallel → Independent models → Average predictions
```

**Boosting:**
```
Train sequentially → Each model corrects previous errors → Weighted combination
```

### How Boosting Works

1. **Start**: Train initial model on data
2. **Identify errors**: Find samples where model performs poorly
3. **Weight**: Give more importance to misclassified samples
4. **Train next model**: Focus on hard examples
5. **Combine**: Weight models by performance
6. **Repeat**: Until reaching desired number of models

### Popular Boosting Algorithms

| Algorithm | Year | Key Features |
|-----------|------|--------------|
| **AdaBoost** | 1996 | Adaptive boosting, weights samples |
| **Gradient Boosting** | 1999 | Uses gradient descent, builds on residuals |
| **XGBoost** | 2014 | Extreme gradient boosting, regularization |
| **LightGBM** | 2017 | Fast, memory efficient, leaf-wise growth |
| **CatBoost** | 2017 | Categorical features, ordered boosting |

### AdaBoost Algorithm

**Step-by-step:**
1. Initialize sample weights: w = 1/N for all samples
2. For each iteration:
   - Train weak learner on weighted data
   - Calculate error rate
   - Calculate model weight (higher for better models)
   - Update sample weights (increase for misclassified)
3. Final prediction: Weighted vote

### Gradient Boosting Algorithm

**Key idea**: Each tree predicts the *residuals* (errors) of previous trees

```
F₀(x) = initial prediction
F₁(x) = F₀(x) + α₁ · h₁(x)  # Add tree predicting residuals
F₂(x) = F₁(x) + α₂ · h₂(x)  # Add tree predicting remaining residuals
...
Fₘ(x) = Final prediction
```

### Why Boosting Works

✓ **Reduces bias**: Focuses on hard examples
✓ **Powerful**: Often best performance on tabular data
✓ **Flexible**: Many tuning options

### Challenges

✗ **Overfitting risk**: Can memorize training data
✗ **Sensitive to noise**: Outliers get high weights
✗ **Sequential**: Cannot parallelize easily
✗ **Requires tuning**: Many hyperparameters

Let's see AdaBoost and Gradient Boosting in action!

In [ ]:
# Boosting Algorithms Implementation

print("=" * 60)
print("BOOSTING ALGORITHMS COMPARISON")
print("=" * 60)

# 1. AdaBoost
print("\n1. AdaBoost Classifier")
print("-" * 60)

adaboost = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),  # Weak learners (stumps)
    n_estimators=100,
    learning_rate=1.0,
    random_state=42,
)

adaboost.fit(X_train, y_train)
ada_train = adaboost.score(X_train, y_train)
ada_test = adaboost.score(X_test, y_test)

print(f"Train Accuracy: {ada_train:.4f}")
print(f"Test Accuracy:  {ada_test:.4f}")
print(f"Overfitting:    {ada_train - ada_test:.4f}")

# 2. Gradient Boosting
print("\n2. Gradient Boosting Classifier")
print("-" * 60)

gb = GradientBoostingClassifier(
    n_estimators=100, learning_rate=0.1, max_depth=3, min_samples_split=5, random_state=42
)

gb.fit(X_train, y_train)
gb_train = gb.score(X_train, y_train)
gb_test = gb.score(X_test, y_test)

print(f"Train Accuracy: {gb_train:.4f}")
print(f"Test Accuracy:  {gb_test:.4f}")
print(f"Overfitting:    {gb_train - gb_test:.4f}")

# 3. Comparison: Bagging vs Boosting
print("\n" + "=" * 60)
print("BAGGING VS BOOSTING COMPARISON")
print("=" * 60)

all_models = pd.DataFrame(
    {
        "Model": ["Single Tree", "Bagging", "Random Forest", "AdaBoost", "Gradient Boosting"],
        "Type": ["Baseline", "Bagging", "Bagging", "Boosting", "Boosting"],
        "Train Acc": [train_score, bag_train, rf_train, ada_train, gb_train],
        "Test Acc": [test_score, bag_test, rf_test, ada_test, gb_test],
        "Overfitting": [
            train_score - test_score,
            bag_train - bag_test,
            rf_train - rf_test,
            ada_train - ada_test,
            gb_train - gb_test,
        ],
    }
)

display(all_models)

# Visualize comprehensive comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Accuracy comparison
x = np.arange(len(all_models))
width = 0.35

axes[0, 0].bar(
    x - width / 2, all_models["Train Acc"], width, label="Train", alpha=0.8, color="skyblue"
)
axes[0, 0].bar(x + width / 2, all_models["Test Acc"], width, label="Test", alpha=0.8, color="coral")
axes[0, 0].set_ylabel("Accuracy", fontweight="bold")
axes[0, 0].set_title("All Ensemble Methods - Accuracy", fontweight="bold")
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(all_models["Model"], rotation=45, ha="right")
axes[0, 0].legend()
axes[0, 0].grid(axis="y", alpha=0.3)

# 2. Test accuracy ranking
sorted_models = all_models.sort_values("Test Acc")
colors_gradient = plt.cm.RdYlGn(sorted_models["Test Acc"])
axes[0, 1].barh(sorted_models["Model"], sorted_models["Test Acc"], color=colors_gradient, alpha=0.8)
axes[0, 1].set_xlabel("Test Accuracy", fontweight="bold")
axes[0, 1].set_title("Ranking by Test Performance", fontweight="bold")
axes[0, 1].grid(axis="x", alpha=0.3)

for i, (model, acc) in enumerate(zip(sorted_models["Model"], sorted_models["Test Acc"])):
    axes[0, 1].text(acc, i, f" {acc:.4f}", va="center")

# 3. Overfitting analysis
colors = ["red" if x > 0.05 else "green" for x in all_models["Overfitting"]]
axes[1, 0].barh(all_models["Model"], all_models["Overfitting"], color=colors, alpha=0.7)
axes[1, 0].axvline(x=0.05, color="orange", linestyle="--", linewidth=2, label="5% threshold")
axes[1, 0].set_xlabel("Overfitting (Train - Test)", fontweight="bold")
axes[1, 0].set_title("Overfitting Analysis", fontweight="bold")
axes[1, 0].legend()
axes[1, 0].grid(axis="x", alpha=0.3)

# 4. Bagging vs Boosting
bagging_models = all_models[all_models["Type"].isin(["Bagging"])]
boosting_models = all_models[all_models["Type"] == "Boosting"]

methods = ["Bagging\n(Avg)", "Boosting\n(Avg)"]
avg_acc = [bagging_models["Test Acc"].mean(), boosting_models["Test Acc"].mean()]
colors_bar = ["#3498db", "#e74c3c"]

bars = axes[1, 1].bar(methods, avg_acc, color=colors_bar, alpha=0.8, edgecolor="black", linewidth=2)
axes[1, 1].set_ylabel("Average Test Accuracy", fontweight="bold")
axes[1, 1].set_title("Bagging vs Boosting Performance", fontweight="bold")
axes[1, 1].set_ylim(0, 1)
axes[1, 1].grid(axis="y", alpha=0.3)

for bar, acc in zip(bars, avg_acc):
    height = bar.get_height()
    axes[1, 1].text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"{acc:.4f}",
        ha="center",
        va="bottom",
        fontsize=12,
        fontweight="bold",
    )

plt.tight_layout()
plt.show()

# Learning curves comparison
print("\n" + "=" * 60)
print("BOOSTING FEATURE IMPORTANCE")
print("=" * 60)

# Gradient Boosting feature importance
gb_importance = pd.DataFrame(
    {"Feature": features, "Importance": gb.feature_importances_}
).sort_values("Importance", ascending=False)

print("\nGradient Boosting Feature Importance:")
print(gb_importance.to_string(index=False))

print("\n💡 Key Insights:")
print("   • Boosting often achieves higher test accuracy")
print("   • AdaBoost uses weak learners (shallow trees)")
print("   • Gradient Boosting more flexible, often performs better")
print("   • Boosting can overfit if not tuned properly")
print("\n✓ Boosting fundamentals demonstrated!")

## 4. XGBoost Deep Dive

**XGBoost** (eXtreme Gradient Boosting) is the most popular ML algorithm for structured data.

### Why XGBoost Dominates

🏆 **Kaggle**: Used in 90%+ of winning solutions  
🚀 **Speed**: 10x faster than sklearn's Gradient Boosting  
🎯 **Performance**: State-of-the-art accuracy  
🔧 **Features**: Built-in regularization, handling missing values, parallel processing  

### Key Innovations

1. **Regularization**: L1 (Lasso) and L2 (Ridge) penalties prevent overfitting
2. **Tree Pruning**: Max depth first, then prune backward
3. **Handling Missing Values**: Learns best direction for missing data
4. **Parallel Processing**: Tree construction parallelized
5. **Cache Optimization**: Efficient memory usage

### XGBoost vs Gradient Boosting

| Feature | sklearn GBM | XGBoost |
|---------|-------------|---------|
| Speed | Slow | 10x faster |
| Regularization | No | Yes (L1 + L2) |
| Missing values | Requires imputation | Handles automatically |
| Parallel | No | Yes |
| Early stopping | No | Yes |
| Custom objectives | No | Yes |

### Important Hyperparameters

**Tree Parameters:**
- `max_depth`: Max tree depth (3-10)
- `min_child_weight`: Minimum samples in leaf (1-10)
- `gamma`: Minimum loss reduction (0-5)

**Boosting Parameters:**
- `n_estimators`: Number of trees (100-1000)
- `learning_rate`: Shrinkage (0.01-0.3)
- `subsample`: Row sampling ratio (0.5-1.0)
- `colsample_bytree`: Column sampling (0.5-1.0)

**Regularization:**
- `reg_alpha`: L1 penalty (0-1)
- `reg_lambda`: L2 penalty (0-1)

### Tuning Strategy

1. **Start**: Fix learning_rate=0.1, tune tree params
2. **Then**: Tune n_estimators with early stopping
3. **Finally**: Lower learning_rate, increase n_estimators

Let's see XGBoost in action!

In [ ]:
# XGBoost Implementation

print("=" * 60)
print("XGBOOST - THE KAGGLE CHAMPION")
print("=" * 60)

if xgb is not None:
    # XGBoost with default parameters
    print("\n1. XGBoost with Default Parameters")
    print("-" * 60)

    xgb_default = xgb.XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False)

    xgb_default.fit(X_train, y_train)
    xgb_default_train = xgb_default.score(X_train, y_train)
    xgb_default_test = xgb_default.score(X_test, y_test)

    print(f"Train Accuracy: {xgb_default_train:.4f}")
    print(f"Test Accuracy:  {xgb_default_test:.4f}")

    # XGBoost with tuned parameters
    print("\n2. XGBoost with Tuned Parameters")
    print("-" * 60)

    xgb_tuned = xgb.XGBClassifier(
        n_estimators=200,
        max_depth=5,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,  # L1 regularization
        reg_lambda=1.0,  # L2 regularization
        random_state=42,
        eval_metric="logloss",
        use_label_encoder=False,
    )

    # Train with early stopping
    xgb_tuned.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

    xgb_tuned_train = xgb_tuned.score(X_train, y_train)
    xgb_tuned_test = xgb_tuned.score(X_test, y_test)

    print(f"Train Accuracy: {xgb_tuned_train:.4f}")
    print(f"Test Accuracy:  {xgb_tuned_test:.4f}")
    print(
        f"Best iteration: {xgb_tuned.best_iteration if hasattr(xgb_tuned, 'best_iteration') else 'N/A'}"
    )

    # Comparison with other methods
    print("\n" + "=" * 60)
    print("COMPREHENSIVE ENSEMBLE COMPARISON")
    print("=" * 60)

    final_comparison = pd.DataFrame(
        {
            "Model": [
                "Single Tree",
                "Random Forest",
                "Gradient Boosting",
                "XGBoost Default",
                "XGBoost Tuned",
            ],
            "Train Acc": [train_score, rf_train, gb_train, xgb_default_train, xgb_tuned_train],
            "Test Acc": [test_score, rf_test, gb_test, xgb_default_test, xgb_tuned_test],
            "Overfitting": [
                train_score - test_score,
                rf_train - rf_test,
                gb_train - gb_test,
                xgb_default_train - xgb_default_test,
                xgb_tuned_train - xgb_tuned_test,
            ],
        }
    )

    display(final_comparison)

    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # Test accuracy comparison
    sorted_final = final_comparison.sort_values("Test Acc")
    colors = plt.cm.RdYlGn(sorted_final["Test Acc"])

    axes[0].barh(
        sorted_final["Model"], sorted_final["Test Acc"], color=colors, alpha=0.8, edgecolor="black"
    )
    axes[0].set_xlabel("Test Accuracy", fontweight="bold")
    axes[0].set_title("Final Model Comparison", fontweight="bold", fontsize=14)
    axes[0].grid(axis="x", alpha=0.3)

    for i, (model, acc) in enumerate(zip(sorted_final["Model"], sorted_final["Test Acc"])):
        axes[0].text(acc, i, f" {acc:.4f}", va="center", fontweight="bold")

    # Feature importance from XGBoost
    xgb_importance = pd.DataFrame(
        {"Feature": features, "Importance": xgb_tuned.feature_importances_}
    ).sort_values("Importance", ascending=True)

    axes[1].barh(
        xgb_importance["Feature"],
        xgb_importance["Importance"],
        color="steelblue",
        alpha=0.8,
        edgecolor="black",
    )
    axes[1].set_xlabel("Importance", fontweight="bold")
    axes[1].set_title("XGBoost Feature Importance", fontweight="bold", fontsize=14)
    axes[1].grid(axis="x", alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Training history (if available)
    if hasattr(xgb_tuned, "evals_result"):
        results = xgb_tuned.evals_result()
        if "validation_0" in results and "logloss" in results["validation_0"]:
            plt.figure(figsize=(10, 6))
            plt.plot(results["validation_0"]["logloss"], label="Test Loss", linewidth=2)
            plt.xlabel("Iteration", fontweight="bold")
            plt.ylabel("Log Loss", fontweight="bold")
            plt.title("XGBoost Training History", fontweight="bold", fontsize=14)
            plt.legend()
            plt.grid(alpha=0.3)
            plt.tight_layout()
            plt.show()

    print("\n💡 Key Insights:")
    print("   • XGBoost typically achieves best performance")
    print("   • Regularization prevents overfitting")
    print("   • Early stopping saves computation")
    print("   • Feature importance aids interpretation")

    print("\n✓ XGBoost demonstrated!")

else:
    print("\n⚠️  XGBoost not installed")
    print("Install with: pip install xgboost")
    print("\nXGBoost is the gold standard for tabular data!")
    print("Used in 90%+ of Kaggle winning solutions")

## 5. LightGBM for Large Datasets

**LightGBM** = Light Gradient Boosting Machine (Microsoft)

### Key Advantages

🚀 **Speed**: 10x faster than XGBoost on large data  
💾 **Memory**: Uses less memory  
📊 **Performance**: Often matches or beats XGBoost  

### Unique Features

1. **Leaf-wise Growth**: Grows trees by best leaf (not level-wise)
2. **Histogram-based**: Bins continuous features
3. **GOSS**: Gradient-based One-Side Sampling
4. **EFB**: Exclusive Feature Bundling

### When to Use LightGBM

✓ Large datasets (>10K rows)  
✓ High-dimensional data (>100 features)  
✓ Speed is critical  
✓ Limited memory  

### XGBoost vs LightGBM

| Aspect | XGBoost | LightGBM |
|--------|---------|----------|
| Tree growth | Level-wise | Leaf-wise |
| Speed | Fast | Faster |
| Memory | Moderate | Lower |
| Small data | Better | Can overfit |
| Large data | Good | Better |

**Pro tip**: For datasets <10K rows, use XGBoost. For larger, try LightGBM!

In [ ]:
# LightGBM Implementation (if available)

print("=" * 60)
print("LIGHTGBM - FAST AND EFFICIENT")
print("=" * 60)

if lgb is not None:
    lgb_model = lgb.LGBMClassifier(
        n_estimators=200, max_depth=5, learning_rate=0.1, num_leaves=31, random_state=42, verbose=-1
    )

    lgb_model.fit(X_train, y_train)
    lgb_test = lgb_model.score(X_test, y_test)

    print(f"LightGBM Test Accuracy: {lgb_test:.4f}")
    print("✓ LightGBM trained successfully!")
else:
    print("⚠️  LightGBM not installed")
    print("Install with: pip install lightgbm")
    lgb_test = None

print("\n" + "=" * 60)
print("ADVANCED BOOSTING LIBRARIES SUMMARY")
print("=" * 60)

summary_data = []
if xgb is not None:
    summary_data.append(
        {
            "Library": "XGBoost",
            "Test Accuracy": xgb_tuned_test,
            "Best For": "General purpose, Kaggle",
        }
    )
if lgb is not None:
    summary_data.append(
        {"Library": "LightGBM", "Test Accuracy": lgb_test, "Best For": "Large datasets, speed"}
    )

if summary_data:
    summary_df = pd.DataFrame(summary_data)
    display(summary_df)

    print("\n💡 Recommendations:")
    print("   • XGBoost: Best all-around choice")
    print("   • LightGBM: Use for large datasets (>100K rows)")
    print("   • CatBoost: Best for categorical features")
else:
    print("\nInstall advanced libraries:")
    print("  pip install xgboost lightgbm catboost")

print("\n✓ Advanced gradient boosting libraries covered!")

## 6. CatBoost for Categorical Data

**CatBoost** = Categorical Boosting (Yandex)

### Key Advantage: Built-in Categorical Handling

**Most libraries**: Require manual encoding (one-hot, label)  
**CatBoost**: Handles categorical features automatically!

### Unique Features

1. **Ordered Boosting**: Prevents target leakage
2. **Categorical Features**: No preprocessing needed
3. **Symmetric Trees**: Faster prediction
4. **Robust**: Handles missing values, outliers

### When to Use CatBoost

✓ Many categorical features  
✓ Don't want to engineer encodings  
✓ Need robust model  
✓ Production deployment  

### Quick Comparison

```python
# Other libraries
X['category'] = LabelEncoder().fit_transform(X['category'])

# CatBoost
model.fit(X, y, cat_features=['category'])  # That's it!
```

**Note**: CatBoost handles categories automatically, making it great for datasets with many categorical columns!

In [ ]:
# CatBoost Implementation (if available)

print("=" * 60)
print("CATBOOST - CATEGORICAL FEATURES EXPERT")
print("=" * 60)

if CatBoostClassifier is not None:
    cat_model = CatBoostClassifier(
        iterations=200, depth=5, learning_rate=0.1, random_seed=42, verbose=False
    )

    cat_model.fit(X_train, y_train)
    cat_test = cat_model.score(X_test, y_test)

    print(f"CatBoost Test Accuracy: {cat_test:.4f}")
    print("✓ CatBoost trained successfully!")
    print("\n💡 CatBoost excels with categorical features")
    print("   Use cat_features parameter for categorical columns")
else:
    print("⚠️  CatBoost not installed")
    print("Install with: pip install catboost")

print("\n✓ CatBoost covered!")

## 7. Stacking and Blending

**Stacking** combines different types of models using a meta-learner.

### How Stacking Works

```
Base Models:
  Model 1 (Random Forest) → Predictions 1
  Model 2 (XGBoost)       → Predictions 2  
  Model 3 (Logistic Reg)  → Predictions 3
                              ↓
                    Meta-Model (combines all)
                              ↓
                      Final Prediction
```

### Stacking vs Blending

| Aspect | Stacking | Blending |
|--------|----------|----------|
| Method | Cross-validation | Holdout set |
| Data usage | All training data | Split data |
| Complexity | Higher | Simpler |
| Performance | Better | Good |
| Overfitting risk | Lower | Higher |

### Why Stacking Works

✓ **Diversity**: Different models make different errors  
✓ **Strength**: Combines best of each model  
✓ **Flexibility**: Can use any meta-learner  
✓ **Performance**: Often best single-model alternative  

### Common Stacking Strategies

1. **Voting**: Simple averaging or majority vote
2. **Weighted Voting**: Weight models by performance
3. **Meta-Model**: Train model on predictions

### Best Practices

1. Use diverse base models (different algorithms)
2. Use cross-validation to generate meta-features
3. Simple meta-model (Logistic Regression, Ridge)
4. Don't overfit meta-model
5. Keep validation set separate

Let's build a stacking ensemble!

In [ ]:
# Stacking and Blending Implementation

print("=" * 60)
print("STACKING ENSEMBLE")
print("=" * 60)

# 1. Voting Classifier (Simple Ensemble)
print("\n1. Voting Classifier (Simple Average)")
print("-" * 60)

voting_clf = VotingClassifier(
    estimators=[
        ("rf", RandomForestClassifier(n_estimators=100, random_state=42)),
        ("gb", GradientBoostingClassifier(n_estimators=100, random_state=42)),
        ("lr", LogisticRegression(random_state=42, max_iter=1000)),
    ],
    voting="soft",  # Use probability averaging
)

voting_clf.fit(X_train, y_train)
voting_test = voting_clf.score(X_test, y_test)

print(f"Voting Classifier Test Accuracy: {voting_test:.4f}")

# 2. Stacking Classifier (Meta-Model)
print("\n2. Stacking Classifier (Meta-Model)")
print("-" * 60)

# Base models (diverse algorithms)
base_models = [
    ("rf", RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)),
    ("gb", GradientBoostingClassifier(n_estimators=100, max_depth=3, random_state=42)),
    ("lr", LogisticRegression(random_state=42, max_iter=1000)),
]

# Meta-model (simple model on base predictions)
meta_model = LogisticRegression(random_state=42, max_iter=1000)

# Stacking classifier
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,  # Use 5-fold CV to generate meta-features
    n_jobs=-1,
)

stacking_clf.fit(X_train, y_train)
stacking_test = stacking_clf.score(X_test, y_test)

print(f"Stacking Classifier Test Accuracy: {stacking_test:.4f}")

# 3. Compare All Ensemble Methods
print("\n" + "=" * 60)
print("FINAL ENSEMBLE COMPARISON")
print("=" * 60)

final_results = {
    "Single Tree": test_score,
    "Random Forest": rf_test,
    "Gradient Boosting": gb_test,
    "Voting Ensemble": voting_test,
    "Stacking Ensemble": stacking_test,
}

# Add XGBoost if available
if xgb is not None:
    final_results["XGBoost"] = xgb_tuned_test

results_final_df = pd.DataFrame(
    {"Model": list(final_results.keys()), "Test Accuracy": list(final_results.values())}
).sort_values("Test Accuracy", ascending=False)

display(results_final_df)

# Visualize final comparison
plt.figure(figsize=(12, 6))
colors = plt.cm.RdYlGn(results_final_df["Test Accuracy"])
bars = plt.barh(
    results_final_df["Model"],
    results_final_df["Test Accuracy"],
    color=colors,
    alpha=0.8,
    edgecolor="black",
    linewidth=2,
)

plt.xlabel("Test Accuracy", fontweight="bold", fontsize=12)
plt.title("Complete Ensemble Methods Comparison", fontweight="bold", fontsize=14)
plt.grid(axis="x", alpha=0.3)

# Add value labels
for i, (model, acc) in enumerate(zip(results_final_df["Model"], results_final_df["Test Accuracy"])):
    plt.text(acc, i, f" {acc:.4f}", va="center", fontweight="bold")

plt.tight_layout()
plt.show()

# Best performer
best_model = results_final_df.iloc[0]
improvement = ((best_model["Test Accuracy"] - test_score) / test_score) * 100

print(f"\n🏆 Best Model: {best_model['Model']}")
print(f"   Accuracy: {best_model['Test Accuracy']:.4f}")
print(f"   Improvement over single tree: {improvement:.1f}%")

print("\n💡 Key Insights:")
print("   • Stacking combines diverse models")
print("   • Voting is simpler, often nearly as good")
print("   • Ensembles typically beat single models")
print("   • Diminishing returns after certain point")

print("\n✓ Stacking and blending demonstrated!")

## 8. Kaggle Competition Strategy

Ensemble methods dominate Kaggle competitions. Here's the winning playbook.

### Typical Kaggle Winning Solution

```
Level 1: Diverse Base Models
├─ XGBoost (multiple configs)
├─ LightGBM (multiple configs)
├─ CatBoost
├─ Neural Networks
└─ Random Forest

Level 2: Stacking
└─ Combine with Ridge/Logistic Regression

Level 3: Blending
└─ Weighted average of best models
```

### Kaggle Best Practices

1. **Feature Engineering** (most important!)
2. **Cross-Validation**: 5-10 fold stratified
3. **Multiple Models**: XGBoost + LightGBM + CatBoost
4. **Hyperparameter Tuning**: Bayesian optimization
5. **Ensemble Diversity**: Different algorithms, features, seeds
6. **Stacking**: 2-3 levels maximum
7. **Blending**: Weight by CV score

### Common Mistakes to Avoid

❌ **Overfitting to leaderboard**: Trust your CV score  
❌ **Too complex ensembles**: Diminishing returns  
❌ **Not enough diversity**: Similar models don't help  
❌ **Ignoring features**: Best ensemble won't beat better features  

### Quick Kaggle Template

```python
# 1. Strong single models
xgb_model = xgb.XGBClassifier(params)
lgb_model = lgb.LGBMClassifier(params)

# 2. Different random seeds
models = []
for seed in [42, 123, 456]:
    model = xgb.XGBClassifier(random_state=seed)
    models.append(model)

# 3. Stack or blend
final_pred = np.mean([m.predict_proba(X) for m in models], axis=0)
```

### Resources

- Kaggle Learn: Free courses
- Past competition solutions: Study winners
- Kaggle Forums: Ask questions

**Pro tip**: Start with Titanic or House Prices competitions!

In [ ]:
# Kaggle-Style Ensemble Example

print("=" * 60)
print("KAGGLE-STYLE ENSEMBLE DEMO")
print("=" * 60)

# Simulate Kaggle approach: Multiple models with different seeds
print("\nBuilding ensemble with different random seeds...")

ensemble_predictions = []

# Train same model with different seeds for diversity
for seed in [42, 123, 456, 789]:
    model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=seed, n_jobs=-1)
    model.fit(X_train, y_train)
    pred_proba = model.predict_proba(X_test)[:, 1]
    ensemble_predictions.append(pred_proba)

# Blend predictions (simple average)
blended_proba = np.mean(ensemble_predictions, axis=0)
blended_pred = (blended_proba >= 0.5).astype(int)
blended_accuracy = accuracy_score(y_test, blended_pred)

# Compare single model vs ensemble
single_model_acc = (
    RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    .fit(X_train, y_train)
    .score(X_test, y_test)
)

print(f"\nSingle Model (seed=42):    {single_model_acc:.4f}")
print(f"Ensemble (4 seeds):        {blended_accuracy:.4f}")
print(f"Improvement:               {(blended_accuracy - single_model_acc):.4f}")

print("\n💡 Even simple ensembles improve performance!")
print("   Kaggle winners use 10-50+ models")

print("\n" + "=" * 60)
print("QUICK KAGGLE WORKFLOW")
print("=" * 60)

workflow = """
1. ✓ Exploratory Data Analysis (EDA)
2. ✓ Feature Engineering (MOST IMPORTANT!)
3. ✓ Baseline Model (Random Forest / XGBoost)
4. ✓ Cross-Validation Setup (Stratified K-Fold)
5. ✓ Hyperparameter Tuning (Grid/Random/Bayesian)
6. ✓ Try Multiple Algorithms
7. ✓ Build Ensemble (Stacking/Blending)
8. ✓ Submit and Iterate

📊 Feature Engineering > Ensembles > Hyperparameter Tuning
"""

print(workflow)

print("✓ Kaggle strategy demonstrated!")

## 9. Hands-On Exercises

Practice ensemble methods to master them!

### Exercise 1: Bagging Comparison
Compare BaggingClassifier with different base estimators:
- Decision Tree
- Logistic Regression
- SVM

Which works best? Why?

### Exercise 2: XGBoost Tuning
Tune XGBoost on the customer_reviews dataset:
- Grid search over: n_estimators, max_depth, learning_rate
- Use early stopping
- Compare with default parameters

### Exercise 3: Build a Stacking Ensemble
Create a stacking ensemble with:
- Base: Random Forest, XGBoost, Logistic Regression
- Meta: Ridge Regression
- Evaluate with cross-validation

### Exercise 4: Kaggle Simulation
1. Load Titanic or House Prices dataset
2. Build 3+ different models
3. Create stacking/blending ensemble
4. Beat single best model by 2%+

### Exercise 5: Feature Importance Comparison
Compare feature importance from:
- Random Forest
- Gradient Boosting
- XGBoost

Do they agree? What does this tell you?

Ready to practice!

In [ ]:
# Exercise Workspace

print("=" * 60)
print("EXERCISES - Master Ensemble Methods!")
print("=" * 60)

# Exercise 1: Bagging Comparison
print("\nExercise 1: Bagging with Different Base Estimators")
print("-" * 60)
print("TODO: Create BaggingClassifier with different base models")
print("TODO: Compare performance\n")

# Exercise 2: XGBoost Tuning
print("Exercise 2: XGBoost Hyperparameter Tuning")
print("-" * 60)
print("TODO: Define parameter grid")
print("TODO: Use GridSearchCV")
print("TODO: Evaluate with cross-validation\n")

# Exercise 3: Stacking Ensemble
print("Exercise 3: Build Custom Stacking Ensemble")
print("-" * 60)
print("TODO: Define base models")
print("TODO: Create StackingClassifier")
print("TODO: Compare with individual models\n")

# Exercise 4: Kaggle Simulation
print("Exercise 4: Kaggle Competition Simulation")
print("-" * 60)
print("TODO: Load competition dataset")
print("TODO: Feature engineering")
print("TODO: Build multiple models")
print("TODO: Create ensemble\n")

# Your code here for all exercises

print("\n" + "=" * 60)
print("TIPS FOR SUCCESS")
print("=" * 60)
print("• Start simple, then add complexity")
print("• Always use cross-validation")
print("• Compare against baseline")
print("• Visualize results")
print("• Document what works")
print("=" * 60)

## 10. Key Takeaways & Next Steps

Congratulations! You've mastered ensemble methods - the key to winning ML competitions!

### What You've Learned

#### 1. **Ensemble Fundamentals**
- ✓ Why ensembles work: diversity + aggregation = better predictions
- ✓ Three main approaches: Bagging, Boosting, Stacking
- ✓ When to use ensembles vs single models

#### 2. **Bagging Methods**
- ✓ Bootstrap aggregating reduces variance
- ✓ Random Forest = Bagging + feature randomization
- ✓ Parallel training, robust to overfitting
- ✓ Feature importance for interpretation

#### 3. **Boosting Methods**
- ✓ AdaBoost: Adaptive boosting with weak learners
- ✓ Gradient Boosting: Sequential error correction
- ✓ Reduces bias, focuses on hard examples
- ✓ Requires careful tuning to avoid overfitting

#### 4. **Advanced Boosting Libraries**
- ✓ **XGBoost**: Regularization, speed, Kaggle champion
- ✓ **LightGBM**: Fast for large datasets, leaf-wise growth
- ✓ **CatBoost**: Automatic categorical handling

#### 5. **Meta-Ensembles**
- ✓ Voting: Simple averaging or majority vote
- ✓ Stacking: Meta-model learns optimal combination
- ✓ Blending: Holdout set predictions
- ✓ Diversity is key to ensemble success

#### 6. **Kaggle Strategies**
- ✓ Feature engineering > Ensembles > Hyperparameters
- ✓ XGBoost + LightGBM + CatBoost standard combo
- ✓ Cross-validation essential
- ✓ Stack 2-3 levels maximum

### Key Insights

> **"Ensemble methods win 90%+ of Kaggle competitions"**

**Performance Hierarchy:**
1. **Feature Engineering** (50%+ improvement possible)
2. **Algorithm Selection** (10-20% improvement)
3. **Hyperparameter Tuning** (5-10% improvement)
4. **Ensembles** (5-15% improvement)

**Best ROI**: Feature engineering + XGBoost!

### Quick Reference Guide

| Algorithm | When to Use | Key Parameters |
|-----------|-------------|----------------|
| **Random Forest** | General purpose, baseline | n_estimators, max_depth |
| **XGBoost** | Tabular data, competitions | learning_rate, max_depth, reg_alpha/lambda |
| **LightGBM** | Large datasets (>100K rows) | num_leaves, learning_rate |
| **CatBoost** | Many categorical features | iterations, depth |
| **Stacking** | Maximum performance | base_models, meta_model |

### Common Pitfalls

1. **Over-ensembling**: Diminishing returns after 5-10 models
2. **Lack of diversity**: Similar models don't help
3. **Overfitting**: Trust CV score, not test score
4. **Ignoring features**: Best ensemble < Good features + simple model
5. **Too complex**: Can't debug or deploy easily

### Real-World Applications

Ensemble methods power:
- **E-commerce**: Recommendation systems, fraud detection
- **Finance**: Credit scoring, risk assessment
- **Healthcare**: Disease prediction, treatment outcomes
- **Marketing**: Customer churn, conversion prediction
- **Autonomous vehicles**: Sensor fusion, decision making

### Production Considerations

**Advantages:**
✓ Superior accuracy
✓ Robust to outliers
✓ Feature importance
✓ Handles missing data (XGBoost/LightGBM)

**Challenges:**
✗ Longer training time
✗ Larger model size
✗ Slower predictions
✗ Harder to interpret
✗ More hyperparameters

**Deploy When:**
- Accuracy is critical
- Can afford compute
- Batch predictions OK
- Model monitoring in place

### Next Steps

#### Module 15: Unsupervised Learning
- Clustering (K-Means, DBSCAN, Hierarchical)
- Dimensionality Reduction (PCA, t-SNE)
- Anomaly Detection
- Customer Segmentation

#### Practice Projects
1. **Titanic** (Kaggle): Classification with ensembles
2. **House Prices** (Kaggle): Regression with XGBoost
3. **Credit Card Fraud**: Imbalanced data with boosting
4. **Customer Segmentation**: Clustering + classification

#### Advanced Topics
- **Deep Ensembles**: Neural network ensembles
- **Snapshot Ensembles**: Single training run, multiple models
- **AutoML**: Automated ensemble creation (Auto-sklearn, H2O)
- **Model Interpretation**: SHAP values for ensembles

### Recommended Practice

Spend **3-4 hours**:
1. Complete all 5 exercises
2. Enter Titanic competition on Kaggle
3. Build production ensemble pipeline
4. Compare all methods on your own dataset

### Resources

**Documentation:**
- [XGBoost Docs](https://xgboost.readthedocs.io/)
- [LightGBM Docs](https://lightgbm.readthedocs.io/)
- [CatBoost Docs](https://catboost.ai/)

**Kaggle:**
- Titanic Competition
- House Prices Competition
- Past competition winning solutions

**Books:**
- "Ensemble Methods for Machine Learning" by Gautam Kunapuli
- "Hands-On Machine Learning" Ch. 7 (Ensemble Learning)

**Tools:**
- SHAP for model interpretation
- MLflow for experiment tracking
- Optuna for hyperparameter optimization

### Final Wisdom

> **"The best model is often an ensemble of good models with good features"**

**Ensemble Philosophy:**
- **Diversity beats individual genius**
- **Aggregate intelligence > single expert**
- **Simple combination often works best**

**For Production:**
1. Start with XGBoost (good default)
2. Add LightGBM if large data
3. Simple voting/averaging if needed
4. Complex stacking only if accuracy critical

**For Competitions:**
1. Feature engineering first
2. Multiple XGBoost/LightGBM/CatBoost
3. Stack everything
4. Blend top models

Keep experimenting, keep ensemble-ing!

---

### Module Complete! 🎉

**Time invested**: ~90 minutes  
**Skills gained**: Kaggle-level ensemble methods  
**Confidence**: Competition-ready!

**Next Module**: `15_unsupervised_learning.ipynb` - Discover patterns without labels!

---

*Built with Claude Code | Module 14: Ensemble Methods*